In [ ]:
# Import TF and check for GPU

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)

# Import required libraries

import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from keras import layers
from tensorflow.keras.models import save_model, load_model
import math
from sklearn.model_selection import train_test_split
import sys
import random 

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Load Test Dataset
# test_shape_dir = '/home/abraham/Documents/ms_proj_shape_lc_gen/data_npy/shape_npy/shape_filled8.npy'
# test_lc_dir = '/home/abraham/Documents/ms_proj_shape_lc_gen/data_npy/lc_npy/lc_dict_8.npy'
test_lc_dir = '../data/data_npy/lc_npy/lc_dict_8.npy'
test_shape_dir  = '../data/data_npy/shape_npy/shape_filled8.npy'


test_lc = np.load(test_lc_dir)
test_shape = np.load(test_shape_dir)
# Check equality of number of dataset
if len(test_lc)==len(test_shape):
    print("test Set: No. of LC = No. of shapes")
else:
    sys.exit("EXIT: testn Set: No. of LC != No. of shapes")

# Normalize the image, convert to opacity map
## Test Set
test_shape = test_shape/np.amax(test_shape)
test_shape_where_0 = np.where(test_shape == 0)
test_shape_where_1 = np.where(test_shape == 1)
test_shape[test_shape_where_0] = 1  # 1 represent the shape (1 opacity)
test_shape[test_shape_where_1] = 0  # 0 represent background (0 opacity)


# Normalize the lightcurves
## - Test Set
test_lc_scaled = np.zeros(test_lc.shape)
for i in np.arange(len(test_lc_scaled)):
    test_lc_scaled[i] = (test_lc[i] - np.amin(test_lc[i]))/(np.amax(test_lc[i]) - np.amin(test_lc[i]))

# Append with ones
## - Test Set
test_lc_scaled_append = np.ones((test_lc.shape[0],120))
for i in np.arange(len(test_lc_scaled)):
    test_lc_scaled_append[i,10:110] = test_lc_scaled[i]
print("Extended the light curves")

# Verification
# Plot - Test LCs
num = 3
fig,ax=plt.subplots(num,2, figsize=(4,3), gridspec_kw={ 'width_ratios': [2,1],
        'wspace': 0.2,'hspace': 0.4})
plt.rcParams['figure.dpi'] = 400

ax[0][1].set_title('Shape',size=10)
ax[0][0].set_title('Light Curve (Test Dataset)',size=10)
ax[num-1][0].set_xlabel('Phase',size=10)
ph = np.linspace(-1,1,len(test_lc_scaled_append[0]))
# advance = 60

i = 0
for i in np.arange(0,num):
    k = random.randint(0, len(test_lc_scaled_append)-1)
    ax[i][1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    if(i<num-1): ax[i][0].tick_params(labelbottom = False, bottom = False)
    img = ax[i][1].imshow(test_shape[k],cmap='inferno')
    plt.colorbar(img)
    ax[i][0].set_ylabel('Flux',size=10)
    ax[i][0].set_ylim(-0.5,1.5)
    ax[i][0].plot(ph,test_lc_scaled_append[k],color = 'tab:red',linewidth='2')
    ax[i][0].grid('on')
    i = i + 1


In [ ]:
# Function to Add noise to Test light curve
def add_noise(test_lc_scaled_append,SNR):
    std_dev = 1/SNR
    test_lc_scaled_append_noise = np.ones(test_lc_scaled_append.shape)
    rng = np.random.default_rng()

    for i in np.arange(len(test_lc_scaled_append)):
        noise_temp = np.random.normal(loc=0.0, scale=std_dev, size=len(test_lc_scaled_append[i]))
        rng.shuffle(noise_temp)
        test_lc_scaled_append_noise[i] = test_lc_scaled_append[i] + noise_temp
    return test_lc_scaled_append_noise


In [ ]:
# Add noise to the light curve
test_lc_SNR500 =add_noise(test_lc_scaled_append,SNR=500)
test_lc_SNR200 =add_noise(test_lc_scaled_append,SNR=200)
test_lc_SNR150 =add_noise(test_lc_scaled_append,SNR=150)
test_lc_SNR100 =add_noise(test_lc_scaled_append,SNR=100)
test_lc_SNR75 =add_noise(test_lc_scaled_append,SNR=75)
test_lc_SNR50 =add_noise(test_lc_scaled_append,SNR=50)
# test_lc_SNR35 =add_noise(test_lc_scaled_append,SNR=35)
# test_lc_SNR20 =add_noise(test_lc_scaled_append,SNR=20)


In [ ]:
# # Function to normalize predicted shape
# def normalize_shape(test_predict_shape):
#     test_predict_shape_normalized = np.zeros(test_predict_shape.shape)
#     for i in np.arange(len(test_predict_shape)):
#         test_predict_shape_normalized[i] = (test_predict_shape[i] - np.amin(test_predict_shape[i]))/(np.amax(test_predict_shape[i]) - np.amin(test_predict_shape[i]))
#     print("Normalized the predicted shape")
#     return test_predict_shape_normalized

# #Load ML model
# tf.keras.backend.clear_session()

# model_SNR = load_model("ml_saved_models/model_sep20_SNR_500_v1/model_sep20_SNR_500_v1.h5")
# results = model_SNR.evaluate(test_lc_SNR500, test_shape, batch_size=128)
# print("SNR = 500. Loss ->",results)
# test_predict_shape_SNR500 = model_SNR.predict(test_lc_SNR500)
# test_predict_shape_SNR500 = normalize_shape(test_predict_shape_SNR500)
# tf.keras.backend.clear_session()
# del model_SNR

# model_SNR = load_model("ml_saved_models/model_sep20_SNR_200_v1/model_sep20_SNR_200_v1.h5")
# results = model_SNR.evaluate(test_lc_SNR200, test_shape, batch_size=128)
# print("SNR = 200. Loss ->",results)
# test_predict_shape_SNR200 = model_SNR.predict(test_lc_SNR200)
# test_predict_shape_SNR200 = normalize_shape(test_predict_shape_SNR200)
# tf.keras.backend.clear_session()
# del model_SNR

# model_SNR = load_model("ml_saved_models/model_sep20_SNR_150_v1/model_sep20_SNR_150_v1.h5")
# results = model_SNR.evaluate(test_lc_SNR150, test_shape, batch_size=128)
# print("SNR = 150. Loss ->",results)
# test_predict_shape_SNR150 = model_SNR.predict(test_lc_SNR150)
# test_predict_shape_SNR150 = normalize_shape(test_predict_shape_SNR150)
# tf.keras.backend.clear_session()
# del model_SNR

# model_SNR = load_model("ml_saved_models/model_sep20_SNR_100_v1/model_sep20_SNR_100_v1.h5")
# results = model_SNR.evaluate(test_lc_SNR100, test_shape, batch_size=128)
# print("SNR = 100. Loss ->",results)
# test_predict_shape_SNR100 = model_SNR.predict(test_lc_SNR100)
# test_predict_shape_SNR100 = normalize_shape(test_predict_shape_SNR100)
# tf.keras.backend.clear_session()
# del model_SNR

# model_SNR = load_model("ml_saved_models/model_sep20_SNR_75_v1/model_sep20_SNR_75_v1.h5")
# results = model_SNR.evaluate(test_lc_SNR75, test_shape, batch_size=128)
# print("SNR = 75. Loss ->",results)
# test_predict_shape_SNR75 = model_SNR.predict(test_lc_SNR75)
# test_predict_shape_SNR75 = normalize_shape(test_predict_shape_SNR75)
# tf.keras.backend.clear_session()
# del model_SNR

# model_SNR = load_model("ml_saved_models/model_sep20_SNR_50_v1/model_sep20_SNR_50_v1.h5")
# results = model_SNR.evaluate(test_lc_SNR50, test_shape, batch_size=128)
# print("SNR = 50. Loss ->",results)
# test_predict_shape_SNR50 = model_SNR.predict(test_lc_SNR50)
# test_predict_shape_SNR50 = normalize_shape(test_predict_shape_SNR50)
# tf.keras.backend.clear_session()
# del model_SNR

# model_SNR = load_model("ml_saved_models/model_sep20_SNR_35_v1/model_sep20_SNR_35_v1.h5")
# results = model_SNR.evaluate(test_lc_SNR35, test_shape, batch_size=128)
# print("SNR = 35. Loss ->",results)
# test_predict_shape_SNR35 = model_SNR.predict(test_lc_SNR35)
# test_predict_shape_SNR35 = normalize_shape(test_predict_shape_SNR35)
# tf.keras.backend.clear_session()
# del model_SNR

# model_SNR = load_model("ml_saved_models/model_sep20_SNR_20_v1/model_sep20_SNR_20_v1.h5")
# results = model_SNR.evaluate(test_lc_SNR20, test_shape, batch_size=128)
# print("SNR = 20. Loss ->",results)
# test_predict_shape_SNR20 = model_SNR.predict(test_lc_SNR20)
# test_predict_shape_SNR20 = normalize_shape(test_predict_shape_SNR20)
# tf.keras.backend.clear_session()
# del model_SNR

In [ ]:
# Experiment 2
# Switch on one MODEL and test on LC with different SNR
# Function to normalize predicted shape
def normalize_shape(test_predict_shape):
    test_predict_shape_normalized = np.zeros(test_predict_shape.shape)
    for i in np.arange(len(test_predict_shape)):
        test_predict_shape_normalized[i] = (test_predict_shape[i] - np.amin(test_predict_shape[i]))/(np.amax(test_predict_shape[i]) - np.amin(test_predict_shape[i]))
    print("Normalized the predicted shape")
    return test_predict_shape_normalized

#Load ML model
tf.keras.backend.clear_session()
model_SNR = load_model("../ml_model/sep22_singleModel_uniform_SNR/model_sep22_unf_50_500_v1.h5")

results = model_SNR.evaluate(test_lc_SNR500, test_shape, batch_size=128)
print("SNR = 500. Loss ->",results)
test_predict_shape_SNR500 = model_SNR.predict(test_lc_SNR500)
test_predict_shape_SNR500 = normalize_shape(test_predict_shape_SNR500)

results = model_SNR.evaluate(test_lc_SNR200, test_shape, batch_size=128)
print("SNR = 200. Loss ->",results)
test_predict_shape_SNR200 = model_SNR.predict(test_lc_SNR200)
test_predict_shape_SNR200 = normalize_shape(test_predict_shape_SNR200)

results = model_SNR.evaluate(test_lc_SNR150, test_shape, batch_size=128)
print("SNR = 150. Loss ->",results)
test_predict_shape_SNR150 = model_SNR.predict(test_lc_SNR150)
test_predict_shape_SNR150 = normalize_shape(test_predict_shape_SNR150)

results = model_SNR.evaluate(test_lc_SNR100, test_shape, batch_size=128)
print("SNR = 100. Loss ->",results)
test_predict_shape_SNR100 = model_SNR.predict(test_lc_SNR100)
test_predict_shape_SNR100 = normalize_shape(test_predict_shape_SNR100)

results = model_SNR.evaluate(test_lc_SNR75, test_shape, batch_size=128)
print("SNR = 75. Loss ->",results)
test_predict_shape_SNR75 = model_SNR.predict(test_lc_SNR75)
test_predict_shape_SNR75 = normalize_shape(test_predict_shape_SNR75)

results = model_SNR.evaluate(test_lc_SNR50, test_shape, batch_size=128)
print("SNR = 50. Loss ->",results)
test_predict_shape_SNR50 = model_SNR.predict(test_lc_SNR50)
test_predict_shape_SNR50 = normalize_shape(test_predict_shape_SNR50)

# results = model_SNR.evaluate(test_lc_SNR35, test_shape, batch_size=128)
# print("SNR = 35. Loss ->",results)
# test_predict_shape_SNR35 = model_SNR.predict(test_lc_SNR35)
# test_predict_shape_SNR35 = normalize_shape(test_predict_shape_SNR35)

# results = model_SNR.evaluate(test_lc_SNR20, test_shape, batch_size=128)
# print("SNR = 20. Loss ->",results)
# test_predict_shape_SNR20 = model_SNR.predict(test_lc_SNR20)
# test_predict_shape_SNR20 = normalize_shape(test_predict_shape_SNR20)


In [ ]:
temp = np.linspace(0, len(test_lc_scaled_append)-1,len(test_lc_scaled_append))
temp

In [ ]:
# Plot predicted shapes
num = 5
fig,ax=plt.subplots(num,7, figsize=(13,5), gridspec_kw={ 'width_ratios': [1,1,1,1,1,1,1,1,1],
        'wspace': 0.1,'hspace': 0.4})
plt.rcParams['figure.dpi'] = 1000
plt.rcParams.update({'font.size': 8})

ax[0][0].set_title('Original',size=8)
ax[0][1].set_title('SNR = 500',size=8)
ax[0][2].set_title('SNR = 200',size=8)
ax[0][3].set_title('SNR = 150',size=8)
ax[0][4].set_title('SNR = 100',size=8)
ax[0][5].set_title('SNR = 75',size=8)
ax[0][6].set_title('SNR = 50',size=8)



temp = np.linspace(0, len(test_lc_scaled_append)-1,len(test_lc_scaled_append))
index_test_array = np.random.choice(temp, size=(num,), replace=False)

for i in range(num):
    k =  int(index_test_array[i]) # random.randint(0, len(test_lc_scaled_append)-1) # i
    print("k = ",k)
    ax[i][0].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][2].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][3].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][4].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][5].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][6].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)

    img = ax[i][0].imshow(test_shape[k],cmap='inferno')
    plt.colorbar(img)  

    img = ax[i][1].imshow(test_predict_shape_SNR500[k],cmap='inferno')
    plt.colorbar(img)   

    img = ax[i][2].imshow(test_predict_shape_SNR200[k],cmap='inferno')
    plt.colorbar(img)   

    img = ax[i][3].imshow(test_predict_shape_SNR150[k],cmap='inferno')
    plt.colorbar(img)     

    img = ax[i][4].imshow(test_predict_shape_SNR100[k],cmap='inferno')
    plt.colorbar(img)  

    img = ax[i][5].imshow(test_predict_shape_SNR75[k],cmap='inferno')
    plt.colorbar(img)     

    img = ax[i][6].imshow(test_predict_shape_SNR50[k],cmap='inferno')
    plt.colorbar(img)

plt.suptitle('Test Dataset Prediction for same shape but with light curve with different SNR')
plt.show()

In [ ]:
# Plot predicted shapes
num = 5
fig,ax=plt.subplots(num,7, figsize=(13,5), gridspec_kw={ 'width_ratios': [1,1,1,1,1,1,1],
        'wspace': 0.1,'hspace': 0.4})
plt.rcParams['figure.dpi'] = 1000
plt.rcParams.update({'font.size': 8})

ax[0][0].set_title('Original',size=8)
ax[0][1].set_title('SNR = 500',size=8)
ax[0][2].set_title('SNR = 200',size=8)
ax[0][3].set_title('SNR = 150',size=8)
ax[0][4].set_title('SNR = 100',size=8)
ax[0][5].set_title('SNR = 75',size=8)
ax[0][6].set_title('SNR = 50',size=8)



temp = np.linspace(0, len(test_lc_scaled_append)-1,len(test_lc_scaled_append))
index_test_array = np.random.choice(temp, size=(num,), replace=False)

for i in range(num):
    k =  int(index_test_array[i]) # random.randint(0, len(test_lc_scaled_append)-1) # i
    print("k = ",k)
    ax[i][0].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][2].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][3].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][4].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][5].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][6].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)


    img = ax[i][0].imshow(test_shape[k],cmap='inferno')
    plt.colorbar(img)  

    img = ax[i][1].imshow(test_predict_shape_SNR500[k],cmap='inferno')
    plt.colorbar(img)   

    img = ax[i][2].imshow(test_predict_shape_SNR200[k],cmap='inferno')
    plt.colorbar(img)   

    img = ax[i][3].imshow(test_predict_shape_SNR150[k],cmap='inferno')
    plt.colorbar(img)     

    img = ax[i][4].imshow(test_predict_shape_SNR100[k],cmap='inferno')
    plt.colorbar(img)  

    img = ax[i][5].imshow(test_predict_shape_SNR75[k],cmap='inferno')
    plt.colorbar(img)     

    img = ax[i][6].imshow(test_predict_shape_SNR50[k],cmap='inferno')
    plt.colorbar(img)

plt.suptitle('Test Resuls of a Single ML Model trained on lightcurves with uniform distribution of SNR from 50 to 500')
plt.show()

In [ ]:
# Plot predicted shapes
num = 5
fig,ax=plt.subplots(num,13, figsize=(26,8), gridspec_kw={'width_ratios': [1,2,1,2,1,2,1,2,1,2,1,2,1],
        'wspace': 0.1,'hspace': 0.4})
plt.rcParams['figure.dpi'] = 1000
plt.rcParams.update({'font.size': 8})

ax[0][0].set_title('Original',size=18)
ax[0][1].set_title('SNR = 500',size=18)
ax[0][2].set_title('SNR = 500',size=18)
ax[0][3].set_title('SNR = 200',size=18)
ax[0][4].set_title('SNR = 200',size=18)
ax[0][5].set_title('SNR = 150',size=18)
ax[0][6].set_title('SNR = 150',size=18)
ax[0][7].set_title('SNR = 100',size=18)
ax[0][8].set_title('SNR = 100',size=18)
ax[0][9].set_title('SNR = 75',size=18)
ax[0][10].set_title('SNR = 75',size=18)
ax[0][11].set_title('SNR = 50',size=18)
ax[0][12].set_title('SNR = 50',size=18)

temp = np.linspace(0, len(test_lc_scaled_append)-1,len(test_lc_scaled_append))
index_test_array = np.random.choice(temp, size=(num,), replace=False)

for i in range(num):
    k =  int(index_test_array[i]) # random.randint(0, len(test_lc_scaled_append)-1) # i
    print("k = ",k)
    ax[i][0].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][2].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][3].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][4].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][5].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][6].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][7].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][8].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][9].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][10].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][11].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][12].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)

    img = ax[i][0].imshow(test_shape[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][1].plot(ph, test_lc_SNR500[k],color = 'tab:red',linewidth='2')
    ax[i][1].grid('on')

    img = ax[i][2].imshow(test_predict_shape_SNR500[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][3].plot(ph, test_lc_SNR200[k],color = 'tab:red',linewidth='2')
    ax[i][3].grid('on')

    img = ax[i][4].imshow(test_predict_shape_SNR200[k],cmap='inferno')
    # plt.colorbar(img)   

    ax[i][5].plot(ph, test_lc_SNR150[k],color = 'tab:red',linewidth='2')
    ax[i][5].grid('on')

    img = ax[i][6].imshow(test_predict_shape_SNR150[k],cmap='inferno')
    # plt.colorbar(img)    


    ax[i][7].plot(ph, test_lc_SNR100[k],color = 'tab:red',linewidth='2')
    ax[i][7].grid('on')

    img = ax[i][8].imshow(test_predict_shape_SNR100[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][9].plot(ph, test_lc_SNR75[k],color = 'tab:red',linewidth='2')
    ax[i][9].grid('on')

    img = ax[i][10].imshow(test_predict_shape_SNR75[k],cmap='inferno')
    # plt.colorbar(img)   

    ax[i][11].plot(ph, test_lc_SNR50[k],color = 'tab:red',linewidth='2')
    ax[i][11].grid('on')

    img = ax[i][12].imshow(test_predict_shape_SNR50[k],cmap='inferno')
    plt.colorbar(img)     

plt.suptitle('Test Resuls of a Single ML Model trained on lightcurves with uniform distribution of SNR from 50 to 500')
plt.show()

In [ ]:
# Plot predicted shapes
num = 5
fig,ax=plt.subplots(num,7, figsize=(13,5), gridspec_kw={ 'width_ratios': [1,2,1,2,1,2,1],
        'wspace': 0.1,'hspace': 0.4})
plt.rcParams['figure.dpi'] = 1000
plt.rcParams.update({'font.size': 8})

ax[0][0].set_title('Original',size=8)
ax[0][1].set_title('SNR = 500',size=8)
ax[0][2].set_title('SNR = 500',size=8)
ax[0][3].set_title('SNR = 200',size=8)
ax[0][4].set_title('SNR = 200',size=8)
ax[0][5].set_title('SNR = 150',size=8)
ax[0][6].set_title('SNR = 150',size=8)

temp = np.linspace(0, len(test_lc_scaled_append)-1,len(test_lc_scaled_append))
index_test_array = np.random.choice(temp, size=(num,), replace=False)

for i in range(num):
    k =  int(index_test_array[i]) # random.randint(0, len(test_lc_scaled_append)-1) # i
    print("k = ",k)
    ax[i][0].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][2].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][3].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][4].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][5].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][6].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)

    img = ax[i][0].imshow(test_shape[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][1].plot(ph, test_lc_SNR500[k],color = 'tab:red',linewidth='2')
    ax[i][1].grid('on')

    img = ax[i][2].imshow(test_predict_shape_SNR500[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][3].plot(ph, test_lc_SNR200[k],color = 'tab:red',linewidth='2')
    ax[i][3].grid('on')

    img = ax[i][4].imshow(test_predict_shape_SNR200[k],cmap='inferno')
    # plt.colorbar(img)   

    ax[i][5].plot(ph, test_lc_SNR150[k],color = 'tab:red',linewidth='2')
    ax[i][5].grid('on')

    img = ax[i][6].imshow(test_predict_shape_SNR150[k],cmap='inferno')
    plt.colorbar(img)    


plt.suptitle('Test Resuls of a Single ML Model trained on lightcurves with uniform distribution of SNR from 50 to 500')
plt.show()
# Plot predicted shapes
num = 5
fig,ax=plt.subplots(num,7, figsize=(13,5), gridspec_kw={ 'width_ratios': [1,2,1,2,1,2,1],
        'wspace': 0.1,'hspace': 0.4})
plt.rcParams['figure.dpi'] = 1000
plt.rcParams.update({'font.size': 8})

ax[0][0].set_title('Original',size=8)
ax[0][1].set_title('SNR = 100',size=8)
ax[0][2].set_title('SNR = 100',size=8)
ax[0][3].set_title('SNR = 75',size=8)
ax[0][4].set_title('SNR = 75',size=8)
ax[0][5].set_title('SNR = 50',size=8)
ax[0][6].set_title('SNR = 50',size=8)


# temp = np.linspace(0, len(test_lc_scaled_append)-1,len(test_lc_scaled_append))
# index_test_array = np.random.choice(temp, size=(num,), replace=False)

for i in range(num):
    k =  int(index_test_array[i]) # random.randint(0, len(test_lc_scaled_append)-1) # i
    print("k = ",k)
    ax[i][0].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][2].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][3].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][4].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][5].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][6].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)

    img = ax[i][0].imshow(test_shape[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][1].plot(ph, test_lc_SNR100[k],color = 'tab:red',linewidth='2')
    ax[i][1].grid('on')

    img = ax[i][2].imshow(test_predict_shape_SNR100[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][3].plot(ph, test_lc_SNR75[k],color = 'tab:red',linewidth='2')
    ax[i][3].grid('on')

    img = ax[i][4].imshow(test_predict_shape_SNR75[k],cmap='inferno')
    # plt.colorbar(img)   

    ax[i][5].plot(ph, test_lc_SNR50[k],color = 'tab:red',linewidth='2')
    ax[i][5].grid('on')

    img = ax[i][6].imshow(test_predict_shape_SNR50[k],cmap='inferno')
    # plt.colorbar(img)     

plt.suptitle('Test Resuls of a Single ML Model trained on lightcurves with uniform distribution of SNR from 50 to 500')
plt.show()

In [ ]:
# Plot predicted shapes
num = 5
fig,ax=plt.subplots(num,7, figsize=(13,5), gridspec_kw={ 'width_ratios': [1,2,1,2,1,2,1],
        'wspace': 0.1,'hspace': 0.4})
plt.rcParams['figure.dpi'] = 1000
plt.rcParams.update({'font.size': 8})

ax[0][0].set_title('Original',size=8)
ax[0][1].set_title('SNR = 100',size=8)
ax[0][2].set_title('SNR = 100',size=8)
ax[0][3].set_title('SNR = 75',size=8)
ax[0][4].set_title('SNR = 75',size=8)
ax[0][5].set_title('SNR = 50',size=8)
ax[0][6].set_title('SNR = 50',size=8)


temp = np.linspace(0, len(test_lc_scaled_append)-1,len(test_lc_scaled_append))
index_test_array = np.random.choice(temp, size=(num,), replace=False)

for i in range(num):
    k =  int(index_test_array[i]) # random.randint(0, len(test_lc_scaled_append)-1) # i
    print("k = ",k)
    ax[i][0].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][2].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][3].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][4].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][5].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    ax[i][6].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)

    img = ax[i][0].imshow(test_shape[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][1].plot(ph, test_lc_SNR100[k],color = 'tab:red',linewidth='2')
    ax[i][1].grid('on')

    img = ax[i][2].imshow(test_predict_shape_SNR100[k],cmap='inferno')
    # plt.colorbar(img)  

    ax[i][3].plot(ph, test_lc_SNR75[k],color = 'tab:red',linewidth='2')
    ax[i][3].grid('on')

    img = ax[i][4].imshow(test_predict_shape_SNR75[k],cmap='inferno')
    # plt.colorbar(img)   

    ax[i][5].plot(ph, test_lc_SNR50[k],color = 'tab:red',linewidth='2')
    ax[i][5].grid('on')

    img = ax[i][6].imshow(test_predict_shape_SNR50[k],cmap='inferno')
    # plt.colorbar(img)     

plt.suptitle('Test Resuls of a Single ML Model trained on lightcurves with uniform distribution of SNR from 50 to 500')
plt.show()